# Pruebas para la generacion de datos de entrenamiento

## Cambios para que funcione con clases

Para exportar se ve que crear una clase para almacenar todas las percepciones es muy util por lo que se procede a modificar todas las funciones anteriores que lo requieran para que funcionen con la clase de percepciones.

#### Librerias

In [1]:
import os
import math
import pickle
from zmqRemoteApi import RemoteAPIClient
from random import randint as rn

#### Creacion de clases

In [2]:
#Class for simulation object
class sim_objects ():
    def __init__(self):
        self.client = RemoteAPIClient()
        self.sim = self.client.getObject('sim')
        self.file = 0
        self.random_increments = []
        self.executed = False
        self.handler_ids = []
        self.sequencies = 1
        self.seq_steps = 2

sim_obj = sim_objects()

# Class to store the different handlers the robot needs to operate in the simulator. 
class handlers_class ():
    def __init__(self):
        self.JRF0 = 0
        self.JRF1 = 0
        #self.JRF2 = 0
        self.JRB0 = 0
        self.JRB1 = 0
        #self.JRB2 = 0
        self.JLF0 = 0
        self.JLF1 = 0
        #self.JLF2 = 0
        self.JLB0 = 0
        self.JLB1 = 0
        #self.JLB2 = 0
        self.base = 0

handler = handlers_class()

#### Funciones de carga del quad y generacion de acciones random

In [3]:
# Function that load the robot in the scene at the beggining of each evaluation.
def load_quad_class():
    # Opening the connection with CoppeliaSim simulator
    sim_obj.client = RemoteAPIClient()
    sim_obj.sim = sim_obj.client.getObject('sim')
    
    # Setting the simulation mode for working in sinchronous mode. The simulation timming is controlled by the Python program and not by the simulation itselfs.
    sim_obj.client.setStepping(True)
    
    handler.base = sim_obj.sim.getObject("/Base")
    #Right-front
    handler.JRF0 = sim_obj.sim.getObject("/JRF0")
    handler.JRF1 = sim_obj.sim.getObject("/JRF1")
    #handler.JRF2 = sim_obj.sim.getObject("/JRF2")
    #Right-back
    handler.JRB0 = sim_obj.sim.getObject("/JRB0")
    handler.JRB1 = sim_obj.sim.getObject("/JRB1")
    #handler.JRB2 = sim_obj.sim.getObject("/JRB2")
    #Left-front
    handler.JLF0 = sim_obj.sim.getObject("/JLF0")
    handler.JLF1 = sim_obj.sim.getObject("/JLF1")
    #handler.JLF2 = sim_obj.sim.getObject("/JLF2")
    #Left-back
    handler.JLB0 = sim_obj.sim.getObject("/JLB0")
    handler.JLB1 = sim_obj.sim.getObject("/JLB1")
    #handler.JLB2 = sim_obj.sim.getObject("/JLB2")
    
    #Stores the different id for every handler starting with the base
    sim_obj.handler_ids = list(handler.__dict__.values())

    sim_obj.sim.startSimulation()

In [4]:
#Generate list of random positions
def rand_gen():
    randomlist = []
    for i in range(len(sim_obj.handler_ids)-1):
        n = rn(-10, 10) * math.pi / 180
        randomlist.append(n)
    sim_obj.random_increments.append(randomlist)

#### Clase y funcion para percepciones

In [5]:

#Class to store all the perceptions
class perceptions ():
    def __init__(self):
        #Initial perceptions
        self.prev_j_positions = []
        self.prev_j_velocities = []
        self.prev_j_l_velocities = []
        self.prev_j_forces = []
        self.prev_base_pos = []
        self.prev_base_ori = []
        self.prev_base_accel = []

        #Resulting perceptions
        self.post_j_positions = []
        self.post_j_velocities = []
        self.post_j_l_velocities = []
        self.post_j_forces = []
        self.post_base_pos = []
        self.post_base_ori = []
        self.post_base_accel = []        

perception = perceptions()

In [6]:

def get_joints_preceptions():
    positions = []
    forces = []
    velocities = []
    l_velocities = []
    base_pos = sim_obj.sim.getObjectPosition(handler.base, sim_obj.sim.handle_world)
    base_ori = sim_obj.sim.getObjectOrientation(handler.base, sim_obj.sim.handle_world)
    base_accel = [0,0,0] #sim_obj.sim.getObjectOrientation(handler.base, sim_obj.sim.handle_world) --- MISSING
    for i in sim_obj.handler_ids[:-1]:
        #Angular/linear position of the joint
        pos = sim_obj.sim.getJointPosition(i)
        positions.append(pos)
        #Torque/force being aplied by the joint
        force = sim_obj.sim.getJointForce(i)
        forces.append(force)
        #Angular/linear velocity of the joint's movement
        veloc = sim_obj.sim.getJointVelocity(i)
        velocities.append(veloc)
        #Linear velocity of the joint
        l_veloc = sim_obj.sim.getJointVelocity(i)
        l_velocities.append(l_veloc)
    if  not sim_obj.executed:
        print("Getting prev perceptions")
        perception.prev_j_positions.append(positions)
        perception.prev_j_velocities.append(velocities)
        perception.prev_j_l_velocities.append(l_velocities)
        perception.prev_j_forces.append(forces)
        perception.prev_base_pos.append(base_pos)
        perception.prev_base_ori.append(base_ori)
        perception.prev_base_accel.append(base_accel)
    else:
        print("Getting post perceptions")
        perception.post_j_positions.append(positions)
        perception.post_j_velocities.append(velocities)
        perception.post_j_l_velocities.append(l_velocities)
        perception.post_j_forces.append(forces)
        perception.post_base_pos.append(base_pos)
        perception.post_base_ori.append(base_ori)
        perception.post_base_accel.append(base_accel)

#### Exportar

In [7]:
def export():    
    #Dump data in file
    sim_obj.file = open(os.getcwd() + r'\training_dataset.txt', 'wb')
    pickle.dump(perception, sim_obj.file)
    sim_obj.file.close()

#### Ejecucion

##### Una ejecucion y un paso (una accion por modulo)

In [19]:
#Main function for one execution
def main():
    print("main function")
    sim_obj.sim.startSimulation()
    load_quad_class()
    rand_gen()
    get_joints_preceptions()
    #Execute every joint action
    print("Executing actions")
    for i in range(len(sim_obj.random_increments[-1])):
        sim_obj.sim.setJointTargetPosition(sim_obj.handler_ids[i], (sim_obj.random_increments[-1][i]+perception.prev_j_positions[-1][i]))
        sim_obj.client.step()
        sim_obj.client.step()
        sim_obj.client.step()
        sim_obj.client.step()
    sim_obj.executed = True
    get_joints_preceptions()
    sim_obj.executed = False
    sim_obj.sim.stopSimulation()
    export()

In [20]:
#main()

print("")

print("Handler ids: ")
print(sim_obj.handler_ids)
print("Joints handler ids: ")
print(sim_obj.handler_ids[:-1])

print("")

data = 0
num_exec = []
for i in perception.__dict__.keys():
    #print(i)
    for k in perception.__dict__[i]:
        #print(k)
        #print("")
        data = data + len(k)
    num_exec.append(len(perception.__dict__[i]))
print(f"Cantidad total de datos para {max(num_exec)} ejecuciones con 1 paso por ejecucion: {data}")
print(f"Datos por ejecucion: {data//max(num_exec)}")


Handler ids: 
[]
Joints handler ids: 
[]

Cantidad total de datos para 0 ejecuciones con 1 paso por ejecucion: 0


ZeroDivisionError: integer division or modulo by zero

##### Una secuencia de multiples pasos (una accion por modulo)

In [27]:
sim_obj.sequencies = 2
print("Numero de secuencias: ",sim_obj.sequencies)
sim_obj.seq_steps = 2
print("Steps por secuencia: ",sim_obj.seq_steps)
perception = perceptions()

Numero de secuencias:  2
Steps por secuencia:  2


In [21]:
#Main function for one execution and multiple steps
def main():
    k = 0
    print("main function")
    sim_obj.sim.startSimulation()
    load_quad_class()
    for j in range(sim_obj.seq_steps):
        print("----------- Secuencia: ", k)
        print("----------- Paso: ", j)
        rand_gen()
        get_joints_preceptions()
        #Execute every joint action
        print(perception.prev_j_positions[-1])
        print("Executing actions")
        for i in range(len(sim_obj.random_increments[-1])):
            sim_obj.sim.setJointTargetPosition(sim_obj.handler_ids[i], (sim_obj.random_increments[-1][i]+perception.prev_j_positions[-1][i]))
            sim_obj.client.step()
            sim_obj.client.step()
            sim_obj.client.step()
            sim_obj.client.step()
        sim_obj.executed = True
        get_joints_preceptions()
        sim_obj.executed = False
        print("")
    print("Simulation has finished")
    sim_obj.sim.stopSimulation()
    #export()

In [ ]:
#if __name__ == "__main__":
#    main()

In [25]:
sim_obj.sim.stopSimulation()
main()

print("")
data = 0
num_exec = []
for i in perception.__dict__.keys():
    #print(i)
    for k in perception.__dict__[i]:
        #print(k)
        #print("")
        data = data + len(k)
    num_exec.append(len(perception.__dict__[i]))
print(f"Cantidad total de datos: {data}")

main function
----------- Secuencia:  0
----------- Paso:  0
Getting prev perceptions
[-0.012899398803710938, 1.565654993057251, -0.24458003044128418, 1.570751428604126, -0.24683666229248047, 1.5711796283721924, -0.2627441883087158, 1.567629098892212]
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  1
Getting prev perceptions
[-0.02315807342529297, 1.636598825454712, -0.3494582176208496, 1.6992723941802979, -0.4849865436553955, 1.5538227558135986, -0.7216746807098389, 1.534343957901001]
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  2
Getting prev perceptions
[-0.10886192321777344, 1.7518866062164307, -0.5547115802764893, 1.5753395557403564, -0.7091200351715088, 1.6534674167633057, -0.8247027397155762, 1.5342772006988525]
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  3
Getting prev perceptions
[-0.0735158920288086, 1.6224005222320557, -0.6642141342163086, 1.49386

##### Multiples secuencias y multiples pasos

In [17]:
sim_obj.sequencies = 2
print("Numero de secuencias: ",sim_obj.sequencies)
sim_obj.seq_steps = 3
print("Steps por secuencia: ",sim_obj.seq_steps)
perception = perceptions()

#Main function for multiple executions and multiple steps
def main():
    print("main function")
    for k in range(sim_obj.sequencies):
        print(f"{k} sequence has started")
        print("")
        #sim_obj.sim.startSimulation()
        load_quad_class()
        for j in range(sim_obj.seq_steps):
            print("----------- Secuencia: ", k)
            print("----------- Paso: ", j)
            rand_gen()
            get_joints_preceptions()
            #Execute every joint action
            print("Executing actions")
            for i in range(len(sim_obj.random_increments[-1])):
                print(sim_obj.random_increments[-1][i]+perception.prev_j_positions[-1][i])
                sim_obj.sim.setJointTargetPosition(sim_obj.handler_ids[i], (sim_obj.random_increments[-1][i]+perception.prev_j_positions[-1][i]))
                sim_obj.client.step()
                sim_obj.client.step()
                sim_obj.client.step()
                sim_obj.client.step()
            sim_obj.executed = True
            get_joints_preceptions()
            sim_obj.executed = False
            print("")
        print(f"{k} simulation has finished")
        print("")
        sim_obj.sim.stopSimulation()
    export()

Numero de secuencias:  2
Steps por secuencia:  3


In [9]:
main()

main function
0 sequence has started

----------- Secuencia:  0
----------- Paso:  0
Getting prev perceptions
Executing actions
0.017453292519943295
1.5009832004265133
-0.17453292519943295
1.5707963705062866
0.17453292519943295
1.7453292957057196
-0.017453292519943295
1.6231562480661166
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  1
Getting prev perceptions
Executing actions
-0.0017066990446014986
1.4168984715586088
-0.3496258433217624
1.5328541209079893
-0.6178447386069948
1.49470349638565
-0.6789245023992766
1.7409236605519869
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  2
Getting prev perceptions
Executing actions
-0.10364353365359531
1.4053490161895752
-0.31580984883847013
1.4929209162571104
-0.7177458390924226
1.6093224257892154
-0.9032126228896489
1.877048666874436
Getting post perceptions

0 simulation has finished

1 sequence has started

----------- Secuencia:  1
----------- Paso:  0
Getting prev perceptions
Executing actions

In [11]:
data = 0
num_exec = []
for i in perception.__dict__.keys():
    #print(i)
    for k in perception.__dict__[i]:
        #print(k)
        #print("")
        data = data + len(k)
    num_exec.append(len(perception.__dict__[i]))
print(f"Cantidad total de datos para {max(num_exec)} secuencias con 1 paso por ejecucion: {data}")
print(f"Datos por ejecucion: {data//max(num_exec)}")

Cantidad total de datos para 6 secuencias con 1 paso por ejecucion: 492
Datos por ejecucion: 82


In [12]:
data = 0
for i in perception.__dict__.keys():
    #print(i)
    for k in perception.__dict__[i]:
        #print(k)
        #print("")
        data = data + len(k)
print(data)

492


##### Pruebas con la funcion de exportar

In [13]:
sim_obj.file = open(os.getcwd() + r'\test.txt', 'wb')
pickle.dump(perception, sim_obj.file)
sim_obj.file.close()
for i in perception.__dict__.keys():
    print(i)
    for k in perception.__dict__[i]:
        print(k)
        print("")

prev_j_positions
[0.0, 1.5707963705062866, 0.0, 1.5707963705062866, 0.0, 1.5707963705062866, 0.0, 1.5707963705062866]

[0.015746593475341797, 1.5041649341583252, -0.2623593807220459, 1.567760705947876, -0.5480315685272217, 1.669236421585083, -0.7312843799591064, 1.6536571979522705]

[0.0010762214660644531, 1.4053490161895752, -0.4205296039581299, 1.527827501296997, -0.6653859615325928, 1.5046026706695557, -0.7635862827301025, 1.7199690341949463]

[-0.015567302703857422, 1.564117670059204, -1.300107717514038, 1.5675451755523682, -1.5694262981414795, 1.5734198093414307, -0.7453248500823975, 1.571265459060669]

[-0.011882543563842773, 1.7214758396148682, -1.445695161819458, 1.3953325748443604, -1.5720198154449463, 1.5542232990264893, -0.7483906745910645, 1.5905191898345947]

[0.059033870697021484, 1.9011986255645752, -1.574479341506958, 1.2481739521026611, -1.5750439167022705, 1.5171244144439697, -0.7485775947570801, 1.4513542652130127]

prev_j_velocities
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

##### Pruebas para la carga de datos

In [14]:
load_file = open(os.getcwd() + r'\test.txt', 'rb')
pickled_prception_class = pickle.load(load_file)
load_file.close()
for i in pickled_prception_class.__dict__.keys():
    print(i)
    for k in pickled_prception_class.__dict__[i]:
        print(k)
        print("")

prev_j_positions
[0.0, 1.5707963705062866, 0.0, 1.5707963705062866, 0.0, 1.5707963705062866, 0.0, 1.5707963705062866]

[0.015746593475341797, 1.5041649341583252, -0.2623593807220459, 1.567760705947876, -0.5480315685272217, 1.669236421585083, -0.7312843799591064, 1.6536571979522705]

[0.0010762214660644531, 1.4053490161895752, -0.4205296039581299, 1.527827501296997, -0.6653859615325928, 1.5046026706695557, -0.7635862827301025, 1.7199690341949463]

[-0.015567302703857422, 1.564117670059204, -1.300107717514038, 1.5675451755523682, -1.5694262981414795, 1.5734198093414307, -0.7453248500823975, 1.571265459060669]

[-0.011882543563842773, 1.7214758396148682, -1.445695161819458, 1.3953325748443604, -1.5720198154449463, 1.5542232990264893, -0.7483906745910645, 1.5905191898345947]

[0.059033870697021484, 1.9011986255645752, -1.574479341506958, 1.2481739521026611, -1.5750439167022705, 1.5171244144439697, -0.7485775947570801, 1.4513542652130127]

prev_j_velocities
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [16]:
if perception.__dict__ == pickled_prception_class.__dict__:
    print("Se conserva el mismo objeto")
else:
    print("No se conserva el mismo objeto")

Se conserva el mismo objeto
